In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from sklearn.metrics import f1_score

# Define paths
train_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train'
train_labels_path = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv'

# Load labels
df = pd.read_csv(train_labels_path)


In [ ]:
Data Transforms and Dataset Class


In [ ]:
# Define transforms for training and validation
image_size = 224
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Custom Dataset Class
class SoilDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'image_id']
        label = self.df.loc[idx, 'label']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
Data Loading and Splitting

In [ ]:
# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Create datasets
train_dataset = SoilDataset(train_df, train_dir, transform=train_transform)
val_dataset = SoilDataset(val_df, train_dir, transform=val_transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
Model Architecture Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained EfficientNet
model = models.efficientnet_b0(pretrained=True)

# Replace classifier for binary classification
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)

model = model.to(device)


In [ ]:
Training Loop Implementation

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            outputs = model(images)
            preds = torch.sigmoid(outputs) > 0.5

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    f1 = f1_score(y_true, y_pred)
    return f1

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_f1 = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Val F1: {val_f1:.4f}")
